In [18]:
# 看一看用的什么显卡
!nvidia-smi

Wed Sep  2 00:23:09 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
!pip install transformers
!unzip /content/drive/Shared\ drives/hbyscgsg@gmail.com/weibo_senti_100k.zip

In [20]:
import torch 
import pandas as pd

In [22]:
pd_all = pd.read_csv("/content/weibo_senti_100k.csv")
print('评论数目（总体）：%d' % pd_all.shape[0])
print('评论数目（正向）：%d' % pd_all[pd_all.label==1].shape[0])
print('评论数目（负向）：%d' % pd_all[pd_all.label==0].shape[0])
pd_all.sample(20)

评论数目（总体）：119988
评论数目（正向）：59993
评论数目（负向）：59995


,label,review
7581,1,欢迎常来哈!![鼓掌] //@江山映像:给自己加加钢:中午和同事溜达到此，本想补补气，甩去这...
19558,1,幸福可以像花儿一样~~办公室的惊喜~~谢谢~~[爱你]
57935,1,[亲亲]参加《优品》活动的演出照片，《优品》杂志只推介高大上，宣称“君子爱奢，侈之有道”的理...
9456,1,兰亭序~王羲之童鞋的啦~[嘻嘻] //@国安密码:高大上的节奏，看不懂啊
114251,0,当你行走他乡的时候，有木有一个导游在唾沫横飞的乱侃着当地香艳的传说故事？抑或是阴森着讲述那不...
36033,1,自从有了微信，每个月的短信基本为零。现在微信也可以视频了。像我这样整天足不出户的宅亲有了和世...
81278,0,原味酸奶，酸......[泪]
83393,0,"wow~帝都的四惠地铁站还是小了点,今儿算是感受到人潮如织的意义啦[汗]检票口到站台拐来拐去..."
116864,0,[泪]哇，真的是第一个~~谢谢樱桃钗小姑凉~~当当当！
86386,0,感觉我跟周伯通似的，左右互搏啊。。。。[泪]


In [2]:
train_size = int(0.8 * 119988)
test_size = 119988 - train_size
train_indices,test_indices = torch.utils.data.random_split(pd_all,[train_size,test_size])

NameError: name 'torch' is not defined

In [99]:
sentences = pd_all['review']
labels = pd_all['label']

In [125]:
train_labels=[]
for i in train_indices.indices:
  train_labels.append(labels[i])

train_sentences=[]
for i in train_indices.indices:
  train_sentences.append(sentences[i])

test_labels=[]
for i in test_indices.indices:
  test_labels.append(labels[i])

test_sentences=[]
for i in test_indices.indices:
  test_sentences.append(sentences[i])

In [123]:
len(train_labels)

95990

In [116]:
from transformers import BertTokenizer
bert_model = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(bert_model)

In [106]:
# Print the original sentence.
print(' Original: ', train_sentences[1])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(train_sentences[1]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train_sentences[1])))

 Original:  #年夜饭# 国贸饭店东方餐厅推出三款不同价位的年夜饭，特别推荐[good]【雪菜明虾球】【曲靖风味大排】【菌菇汁牛柳】，邀您与家人一起来品尝这些美味佳肴！[围观][太开心]
Tokenized:  ['#', '年', '夜', '饭', '#', '国', '贸', '饭', '店', '东', '方', '餐', '厅', '推', '出', '三', '款', '不', '同', '价', '位', '的', '年', '夜', '饭', '，', '特', '别', '推', '荐', '[', 'good', ']', '【', '雪', '菜', '明', '虾', '球', '】', '【', '曲', '靖', '风', '味', '大', '排', '】', '【', '菌', '菇', '汁', '牛', '柳', '】', '，', '邀', '您', '与', '家', '人', '一', '起', '来', '品', '尝', '这', '些', '美', '味', '佳', '肴', '！', '[', '围', '观', ']', '[', '太', '开', '心', ']']
Token IDs:  [108, 2399, 1915, 7649, 108, 1744, 6588, 7649, 2421, 691, 3175, 7623, 1324, 2972, 1139, 676, 3621, 679, 1398, 817, 855, 4638, 2399, 1915, 7649, 8024, 4294, 1166, 2972, 5773, 138, 9005, 140, 523, 7434, 5831, 3209, 6007, 4413, 524, 523, 3289, 7473, 7599, 1456, 1920, 2961, 524, 523, 5826, 5823, 3723, 4281, 3394, 524, 8024, 6913, 2644, 680, 2157, 782, 671, 6629, 3341, 1501, 2214, 6821, 763, 5401, 1456, 881, 5510, 8013, 138, 1741, 6225, 140, 138, 1922, 2458, 2552,

In [117]:
MAX_LEN=128

input_ids = [tokenizer.encode(sent,add_special_tokens=True,max_length=MAX_LEN,truncation=True) for sent in train_sentences]
test_input_ids = [tokenizer.encode(sent,add_special_tokens=True,max_length=MAX_LEN,truncation=True) for sent in test_sentences]

from keras.preprocessing.sequence import pad_sequences
print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")
print("finish")


Padding token: "[PAD]", ID: 0
finish


In [119]:
 # Create attention masks
attention_masks = []

# For each sentence...
for sent in input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

    
test_attention_masks = []

# For each sentence...
for sent in test_input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]
    test_attention_masks.append(att_mask)


In [126]:
from sklearn.model_selection import train_test_split
labels = train_labels
# Use 90% for training and 10% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2020, test_size=0.1)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2020, test_size=0.1)

In [133]:
import torch
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
test_inputs=torch.tensor(test_input_ids)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
test_labels=torch.tensor(test_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)
test_masks=torch.tensor(test_attention_masks)


from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.

batch_size = 16

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# Create the DataLoader for our test set.
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to u

In [128]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-chinese", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [129]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 2

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [130]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [132]:
import random

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, 1):# epoches
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):

        if step % 40 == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)

            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():        
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 2 ========
Training...
  Batch    40  of  5,400.    Elapsed: 0:00:17.
  Batch    80  of  5,400.    Elapsed: 0:00:34.
  Batch   120  of  5,400.    Elapsed: 0:00:51.
  Batch   160  of  5,400.    Elapsed: 0:01:08.
  Batch   200  of  5,400.    Elapsed: 0:01:25.
  Batch   240  of  5,400.    Elapsed: 0:01:42.
  Batch   280  of  5,400.    Elapsed: 0:01:59.
  Batch   320  of  5,400.    Elapsed: 0:02:17.
  Batch   360  of  5,400.    Elapsed: 0:02:34.
  Batch   400  of  5,400.    Elapsed: 0:02:51.
  Batch   440  of  5,400.    Elapsed: 0:03:08.
  Batch   480  of  5,400.    Elapsed: 0:03:26.
  Batch   520  of  5,400.    Elapsed: 0:03:43.
  Batch   560  of  5,400.    Elapsed: 0:04:00.
  Batch   600  of  5,400.    Elapsed: 0:04:18.
  Batch   640  of  5,400.    Elapsed: 0:04:35.
  Batch   680  of  5,400.    Elapsed: 0:04:52.
  Batch   720  of  5,400.    Elapsed: 0:05:09.
  Batch   760  of  5,400.    Elapsed: 0:05:27.
  Batch   800  of  5,400.    Elapsed: 0:05:44.
  Batch   840  of

In [134]:
t0 = time.time()
model.eval()

# Tracking variables 
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# Evaluate data for one epoch
for batch in test_dataloader:
    
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():        
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # Get the "logits" output by the model. The "logits" are the output
    # values prior to applying an activation function like the softmax.
    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # Calculate the accuracy for this batch of test sentences.
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    # Accumulate the total accuracy.
    eval_accuracy += tmp_eval_accuracy

    # Track the number of batches
    nb_eval_steps += 1
print("  Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
print("  Test took: {:}".format(format_time(time.time() - t0)))

  Accuracy: 0.9757
  Test took: 0:03:14
